In [1]:
import sys
import os
ROOT_DIR = os.path.abspath("/MoStress/")
sys.path.append(ROOT_DIR)

In [2]:
import json
from os.path import join

import numpy as np
import pandas as pd
from kerasbeats import prep_time_series
from utils.preprocessingCheckpoint import getPreprocessingCheckpoint
from models.architectures.NBeats.NBeatsModel import NBeatsModel
from tensorflow import keras
import tensorflow as tf


In [3]:
trainData, validationData = getPreprocessingCheckpoint()

In [4]:
dataset = {
    "features": trainData["features"],
    "targets": trainData["targets"],
    "weights": trainData["weights"],
}

In [5]:
inputTest = dataset["features"][0]

inputTest

array([[-0.04906158,  0.32236075,  2.31629804, -0.70362568,  0.35170601],
       [ 0.00304426, -0.309945  ,  2.28104558, -0.45453641,  0.43153586],
       [ 0.62631692,  0.45954869,  2.19640463, -0.31583771,  0.497078  ],
       ...,
       [-0.00608964, -0.49451708,  1.34191815, -2.03553343, -0.74628732],
       [ 0.23747035,  0.34576267,  1.38512626, -1.23622251, -0.75530661],
       [-2.68725794,  1.26433206,  1.45325614, -2.09487152, -0.75049123]])

In [ ]:
X, y = prep_time_series(signalTest, lookback = 7, horizon = 1)

In [ ]:
X.shape, y.shape

In [ ]:
nBeats = NBeatsModel()

In [ ]:
nBeats.fit(X, y, epochs = 10, verbose = True)

In [ ]:
predictions = nBeats.predict(X)
predictions

In [ ]:
residuals = nBeats.residualModel.predict(X)
residuals

In [ ]:
len(residuals), residuals[0].shape

In [ ]:
if tf.config.list_physical_devices('GPU'):
  print("TensorFlow **IS** using the GPU")
else:
  print("TensorFlow **IS NOT** using the GPU")